In [1]:
import numpy as np

*softmax function* <br>
the output of each neure is connected with all the imput


In [2]:
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y

**Warning** exp() may cause calculation overflow <br>
so we can add an costant in the exp() like _exp(a + C)_

In [3]:
# example of overflow
a = np.array([1010, 1000, 990])
y = softmax(a)
print(y)

[nan nan nan]


C:\Users\wq\AppData\Local\Temp\ipykernel_23032\680135705.py:2: RuntimeWarning: overflow encountered in exp
  exp_a = np.exp(a)
C:\Users\wq\AppData\Local\Temp\ipykernel_23032\680135705.py:4: RuntimeWarning: invalid value encountered in divide
  y = exp_a / sum_exp_a


In [4]:
# to deal with the overflow
# we usually substract the maximized input
def softmax(a):
    C = np.max(a)
    exp_a = np.exp(a - C)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y

a = np.array([1010, 1000, 990])
y = softmax(a)
print(y)

[9.99954600e-01 4.53978686e-05 2.06106005e-09]


**the overflow problem has been solved!!!** <br>
specially, the sum of output processed by softmax is always **1** <br>
so we can recognize each output as the probability

In [5]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist
(X_train, t_train), (X_test, t_test) = load_mnist(flatten=True, normalize=False)

print(X_train.shape)
print(X_test.shape)
print(t_train.shape)
print(t_test.shape)

(60000, 784)
(10000, 784)
(60000,)
(10000,)


### highlight! <br>
encounter 403 error, fail to use original code to download the dataset <br>
***solution:*** directly download the dataset and revise the original code successfully <br>
Brilliant!!!

In [7]:
# now we try to show the image of mnist
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
img = x_train[0]
label = t_train[0]
print(label)

print(img.shape)
img = img.reshape(28,28)
print(img.shape)

img_show(img)

5
(784,)
(28, 28)


### realize the inference processing of neural network <br>
the structure of NN <br>
3 layers <br>
input layer: 784 neures (_the size of the image is 28*28=784_) <br>
hidden layer1: 50 neures <br>
hidden layer2: 100 neures

In [8]:
import pickle
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)

        return network
    
def predict(network, x):
    w1, w2, w3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, w1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, w2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, w3) + b3
    y = softmax(a3)

    return y

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


In [9]:
# appraise the accuracy of the NN
x, t = get_data()
net_work = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(net_work, x[i])
    p = np.argmax(y)
    if p == t[i]:
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt / len(x))))


Accuracy:0.9352


In [10]:
# it's faster to process the data in a batch
x, t = get_data()
net_work = init_network()

batch_size = 100
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i: i+batch_size]
    y_batch = predict(net_work, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i: i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt / len(x))))


Accuracy:0.9352
